# Code Inspector Agent

A simple example demonstrating how to build an agent that can search through codebases and answer questions about code structure. This agent uses custom actions to search code repositories and analyze findings, allowing developers to quickly understand unfamiliar codebases.

First, index the codebase in a vector store.

Set `repo_dir` to the path of the repository you want to index.

In [1]:
from moatless.index import CodeIndex, IndexSettings
from moatless.repository import FileRepository
from moatless.workspace import Workspace

import os
import logging
from dotenv import load_dotenv

#logging.basicConfig(level=logging.WARNING)
#logging.getLogger("moatless").setLevel(logging.INFO)

load_dotenv()

# An OPENAI_API_KEY is required to use the OpenAI Models
index_settings = IndexSettings(
    embed_model="text-embedding-3-small",
)


repo_dir = "/tmp/moatless-tools"
persist_dir = "/tmp/moatless-tools/.index"

file_repo = FileRepository(repo_path=repo_dir)

if os.path.exists(persist_dir):
    print(f"Found existing index on {persist_dir}")
    code_index = CodeIndex.from_persist_dir(persist_dir, file_repo=file_repo)
else:
    code_index = CodeIndex(file_repo=file_repo, settings=index_settings)
    nodes, tokens = code_index.run_ingestion()
    print(f"Indexed {nodes} nodes and {tokens} tokens")
    code_index.persist(persist_dir)

workspace = Workspace(
    repository=file_repo,
    code_index=code_index,
    shadow_mode=True # No changes are persisted to the file system
)

Found existing index on /tmp/moatless-tools/.index


The Code Inspector Agent combines a deterministic language model with specialized code analysis actions to create a read-only code exploration tool.

The model uses GPT-4.1-mini with temperature set to 0 for consistent, focused responses, while the prompt guides the agent through a systematic workflow - understanding requests, locating relevant code, gathering details, and reporting findings concisely.

The action toolkit provides all necessary capabilities for thorough code analysis:
* `Think()` enables private reasoning
* `SemanticSearch()` finds relevant code sections using semantic search to the vector store
* `FindClass()` and `FindFunction()` locate specific components
* `FindCodeSnippet()` identifies code patterns
* `ReadFile()` examines file contents
* `Respond()` delivers findings to users

In [2]:
from moatless.actions import ReadFile, FindClass, FindFunction, FindCodeSnippet, StringReplace, CreateFile, Finish, SemanticSearch
from moatless.actions.respond import Respond
from moatless.actions.think import Think
from moatless.completion import ToolCallCompletionModel
from moatless.flow.loop import AgenticLoop
from moatless.message_history import MessageHistoryGenerator
from moatless.agent import ActionAgent

completion_model = ToolCallCompletionModel(
    model="gpt-4.1-mini",
    temperature=0.0,
    model_api_key=""
)

system_prompt = """# Code Inspector Agent Prompt

You are a specialized AI agent whose sole purpose is to explore a codebase and report findings. You cannot modify any code or communicate directly with the user—only return information based on built-in inspection functions.

## 1. Internal Reasoning
- Before starting any inspection task or when facing ambiguity, record a private chain of thought—do not expose it to the user.

## 2. Investigation Workflow
1. **Understand Request**  
   Read the user’s query to determine which parts of the codebase to inspect.  
2. **Locate Code**  
   Use code-search functions to find relevant files, classes, functions, or snippets.  
3. **Gather Details**  
   Open and read the identified code to extract requested information (signatures, dependencies, comments, complexity, etc.).  
4. **Report Findings**  
   Summarize the results clearly and concisely, quoting file paths and code excerpts as needed. Use the `Respond` tool to return the findings.

## 3. Guidelines
- **Read-Only**: Do not alter or write any files—only inspect and report.  
- **Precision**: Provide exact file names, line numbers, and code snippets.  
- **Context**: When relevant, include surrounding code or module relationships.  
- **Brevity**: Keep responses focused on the user’s question.  
- **Tool Usage**: Always use tools, respond by using the `Respond` tool.
"""

agent = ActionAgent(
    completion_model=completion_model,
    system_prompt=system_prompt,
    actions=[
        Think(),
        SemanticSearch(),
        FindClass(),
        FindFunction(),
        FindCodeSnippet(),
        ReadFile(),
        Respond(),
    ],
    memory=MessageHistoryGenerator(),
)

loop = AgenticLoop.create(agent=agent, max_iterations=20)

last_node = await loop.run("Return a detailed description of the agent implementation and what dependencies it has", workspace=workspace)
print(last_node.observation.message)


The agent implementation is primarily in the class `ActionAgent` located in `moatless/agent/agent.py`.

### Description of `ActionAgent`:
- Subclass of `MoatlessComponent`.
- Manages a set of `Action` instances, a `BaseCompletionModel` for generating completions, and a `BaseMemory` for message history.
- Has attributes like `agent_id`, `model_id`, `description`, `system_prompt`, and internal mappings for actions.
- Initialization sets up the completion model with response schemas derived from the actions.
- Provides an async `run` method to process a `Node`, generating and executing action steps.
- Generates actions by creating messages from memory and workspace context, then calling the completion model.
- Executes action steps by mapping action arguments to actual `Action` instances and running them.
- Emits events such as `RunAgentEvent`, `ActionCreatedEvent`, `ActionExecutedEvent`, and `AgentErrorEvent` during its lifecycle.
- Supports workspace setting and initialization of action